# Simulate and estimate univariate time series models (TS)

The goal is to get acquainted with univariate TS models and tests in a controlled envirnoment (=simulated data). Since we controll the environment (=true data-generating process is known) we abstract from practical faliures (e.g. outliers, breaks, model-uninformative data). Instead, we inspect properties of TS models and gain intuition.        

## Packages
We will be using NumPy, MatPlotLib and StatsModels packages. 

NumPy package provides mathematical functions and convenient data types.  
https://numpy.org/doc/stable/user/index.html#user

MatPlotLib provides functions for plotting.  
https://matplotlib.org/stable/users/index

StatsModels package provides standard timer series models and tests.  
https://www.statsmodels.org/stable/user-guide.html 

## Ex.1 Simulate AR(1) data

Simulate data from an AR(1) model by using StatsModels built-in functions. Vary the value of the autoregressive parameter ($\phi_1 \in {0,0.5,0.9,1.0}$). Plot the series and the series autocorrelation (ACF) and partial autocorrelation functions (PACF) for lags 1 to 10.  
 
1. How does the pattern of the series change when persistence increases ($phi_1$&uarr;)?  
2. What happens to the AC and PACF? 
3. Can you tell, from inspecting AC and PACF, which time series model (AR,MA,ARMA) is appropriate for the simulated series?  

In [ ]:
# initialize packages
import numpy  as np
import matplotlib.pyplot as plt
import statsmodels.api as sm

### SIMULATE AND PLOT OBSERVATIONS FROM AN AR(1) MODEL
# TO-DOs: 
# - First, finish the code. 
# - Simulate data for a model with constant term equal to 0 and vary phi. 
#   How does this affect the persistence of the plotted series? 
# - Now set phi = .5 and set constant term to 2. 
#   What will this do? At around what value will the series cycle around? 2? 
# set burn-in observations (to wash out the effect of initial values) and the number of observations to be recorded
nburn = 100
nobs  = 100

# set coeffcient values
c   = 0
phi = 0

# empty place-holder for the AR series 
y = np.zeros(nburn+nobs)

# draw white-noise errors (N~(0,sig2_e??))
e = np.random.standard_normal(nburn+nobs)

# simulate AR(1) process
for i in range(1,nburn+nobs):
    y[i] = c + phi*y[i-1] + e[i]

# drop burn-in observations
y = y[nburn:]

# plot seris, AC and PAC
fig, axs = plt.subplots(1, 3,figsize=(20,10))
axs[0].plot(y)
axs[0].axhline(0,-6,6, color='red')
axs[0].set_title(f"AR(1), phi1={phi1}")
sm.graphics.tsa.plot_acf(y, lags=10, ax=axs[1])
print()
sm.graphics.tsa.plot_pacf(y, lags=10, ax=axs[2])
print()




In [ ]:
## COMPARE THEORETICAL WITH SIMULATED-DATA MOMENTS
# Calculate theoretical mean and variance of y
# and compare them to mean and variance of simulated data:
# TODOs: 
# - Derive the mean and variance of y that distributes as an AR(1) process with a constant. Fill in the below formulas. 
# - Compare the estimated and the theoretical means and variances of AR(1). 
# - What does the below tell you about estimating the true values of model parameters from sample data?  
sig2_e = 1 
mu  = c/(1-phi)
sig2_y   = sig2_e/(1-np.square(phi))
print(f"Theoretical mean is {mu}")
print(f"Sample mean is {np.mean(y)}")
print(f"Theoretical variance is sig^2/(1-phi_1^2) = {sig2_y}")
print(f"Sample variance is {np.var(y)}")

## Ex.2 Simulate MA(1) data

Repeat Ex.1 for an MA(1) model with $\theta_1 \in {0,0.5,0.9,1.0}$. Use StatsModels built-in function for simulating data from ARMA models. 

In [ ]:
# set the number of observations
nobs  = 1000

# set AR and MA lag polynomials 
# format: phi(L) y_t = theta(L) epsilon_t ...
#  (1 + phi1 L + phi2 L^2 +...) y_t = epsilon_t (1 + psi1 L + psi2 L^2 + ...) 
# np.r_ helps quickly concatenating scalars
phiL   = np.r_[1] 
thetaL = np.r_[1,.5]

# simulate y_t
y    = sm.tsa.arma_generate_sample(phiL,thetaL, nobs) 

# plot AC and PAC functions
fig, axs = plt.subplots(1, 3,figsize=(12,10))
axs[0].plot(y);
axs[0].set_title("MA(0,1)");
sm.graphics.tsa.plot_acf(y, lags=10, ax=axs[1]);
sm.graphics.tsa.plot_pacf(y, lags=10, ax=axs[2]);


## Ex.3 Simulate ARMA(1,1) data and perform statistical test

1. Simulate 200 data points from a (stationary!) ARMA(2,0) model. Assume $\phi_1 = 0.6, \phi_2 = .3$. Plot the data.
2. Test the data for stationarity by using augmented Dickey-Fuller and Kwiatkowski-Phillips-Schmidt-Shin test. 
3. Inspect AC and PAC function to select number of AR and MA lags. 
4. Estimate the true model (ARMA(2,0))
5. Test for whitness of the residuals by using the BP autocorrelation test.   
6. Estimate ARMA(1,0), ARMA(3,0) and discuss the differences
7. Display information criteria for AR(1)-AR(3)

In [ ]:
# PART 1
# set the number of observations
nobs  = 1000
# set AR and MA lag polynomials 
phiL   = np.r_[1,-.6,-.3] 
thetaL = np.r_[1]
# simulate y_t
y    = sm.tsa.arma_generate_sample(phiL,thetaL, nobs) 
# plot
plt.plot(y)

In [ ]:
# PART 2: ADF test
# What is null hypothesis in ADF-test? 
print("Results of Dickey-Fuller Test:")
res = sm.tsa.stattools.adfuller(y,maxlag=5,regression='c',autolag='AIC')
print(f"Test Statistic:{res[0]}")
print(f"p-value:{res[1]}")
print(f"#Lags Used:{res[2]}")
print(f"#Observations Used:{res[3]}")
# print(f"#Critical value:{res[4]}")
print("-----------------------------------------------------------------------------------------------------------")
print("DECISION RULE: If p-value is < than desired significance level (1%,5% or 10%) we reject the null hypothesis.")
print("What is the null hypothesis in ADF test?")
print("How do we proceede if the null hypothesis is rejected?")
print("How do we proceede if the null hypothesis is not rejected? Hint: 1) Use differences. 2) For FCS purposes you can in principle continue to use levels. What are the consqeuences in this case?")

In [ ]:
# PART 2: KPSS test
# What is null hypothesis in KPSS-test? 
print("Results of KPSS Test:")
res = sm.tsa.stattools.kpss(y,regression='c',nlags='auto')
print(f"Test Statistic:{res[0]}")
print(f"p-value:{res[1]}")
# print(f"#truncation lags:{res[2]}")
# print(f"#Critical value:{res[3]}")
print("-----------------------------------------------------------------------------------------------------------")
print("DECISION RULE: If p-value is < than desired significance level (1%,5% or 10%) we reject the null hypothesis.")
print("What is the null hypothesis in KPSS test?")
print("What does this test imply?")


In [ ]:
# PART 3 Inspect AC and PAC function and select the number of AR and MA lags. 
fig, axs = plt.subplots(1, 2,figsize=(12,10))
sm.graphics.tsa.plot_acf(y, lags=10, ax=axs[0])
sm.graphics.tsa.plot_pacf(y, lags=10, ax=axs[1])
print("If the results in Ex. (1-3) are unclear, increase the number of observations.")
print("Which model do AC&PAC imply?")


In [ ]:
# PART 4 Estimate the true model
mod = sm.tsa.arima.ARIMA(y, order=(2, 0, 0), trend='c', validate_specification=True)
res = mod.fit()
print(res.summary())
print("---------------------------------------------------------------------------")
print("Interpret statistical significance of the coefficients. E.g. P-values for ar.L1?")
print("Explain LB-test for residual autocorrelation. [See next task..]")
print("What is the null hypothesis in the heteroscedasticity test? What are the consequences in the presence of heteroscedasticity? [SSR in 1st 1/3 of the sample is the same as in the last 1/3 sample.]")
print("What is the null hypothesis in Jarque-Bera test?")



In [ ]:
# PART 5 Test for whitness of the residuals
# First plot the residuals
y_hat = res.fittedvalues
e     = y - y_hat
fig, axs = plt.subplots(1, 2,figsize=(12,10))
axs[0].plot(e)
axs[0].set_title("Residuals")
axs[1].hist(e,bins=100)
axs[1].set_title("Histogram")
# or use >plt.plot(res.resid)
print("The residuals should exhibit no patterns. Especially autocorrelation and/or heteroscedasticity.")
# Perform LB and BP tests for whitness of the residuals
print("Ljung-Box and Box-Pierce for residual autocorrelation")
res_lb = sm.stats.diagnostic.acorr_ljungbox(e, lags=10, boxpierce=True)
print(res_lb)
# rule-of-thumb for number of lags min(10, T/5), https://robjhyndman.com/hyndsight/ljung-box-test/ 
print("Despite wide use, LB test has no asymptotic justification in time series models.")


In [ ]:
# PART 6 Estimate ARMA(1,0)
# ARMA(1,0) - missing an important regressor 
mod = sm.tsa.arima.ARIMA(y, order=(1, 0, 0), trend='c', validate_specification=True)
res = mod.fit()
print(res.summary())
print("---------------------------------------------------------------------------")
print("We estimated an under-fitted model.")
print("Are the residuals uncorrelated? Check the LB test for the first lag in the regression table.")
print("Are the residuals normal? Check the JB test.")
print("Run the LB test for higher lags:")
res_lb = sm.stats.diagnostic.acorr_ljungbox(res.resid, lags=10, boxpierce=True,model_df=2+0)
print(res_lb)
print("Plot the residuals")
fig, axs = plt.subplots(1, 2,figsize=(12,10))
axs[0].plot(res.resid)
axs[0].set_title("Residuals")
axs[1].hist(res.resid,bins=100)
axs[1].set_title("Histogram")

In [ ]:
# PART 6 Estimate ARMA(3,0)
# ARMA(1,0) - missing an important regressor 
mod = sm.tsa.arima.ARIMA(y, order=(3, 0, 0), trend='c', validate_specification=True)
res = mod.fit()
print(res.summary())
print("---------------------------------------------------------------------------")
print("We estimated an under-fitted model.")
print("Are the residuals uncorrelated? Check the LB test for the first lag in the regression table.")
print("Are the residuals normal? Check the JB test.")
print("Run the LB test for higher lags:")
res_lb = sm.stats.diagnostic.acorr_ljungbox(res.resid, lags=10, boxpierce=True,model_df=2+0)
print(res_lb)
print("Plot residuals")
fig, axs = plt.subplots(1, 2,figsize=(12,10))
axs[0].plot(res.resid)
axs[0].set_title("Residuals")
axs[1].hist(res.resid,bins=100)
axs[1].set_title("Histogram")

In [ ]:
# PART 8 Information criteria
# arima_order_select_ic uses ML which issues a lot of redundant warnings so we suppress those
import warnings
warnings.filterwarnings("ignore") 
 

# Estimate AR(1)-AR(3) and display information criteria
# estimate ICs
IC = sm.tsa.stattools.arma_order_select_ic(y, max_ar=5, max_ma=5, ic=['aic','bic','hqic'], trend='c')
# display optimal lags
print(f"min AIC at (p,q): {IC.aic_min_order}")
print(f"min BIC at (p,q): {IC.bic_min_order}")
print(f"min HQIC at (p,q): {IC.hqic_min_order}")
    